In [ ]:
#general analysis packages
import numpy as np
#directory management
import os
#plotting packages
import matplotlib.pyplot as plt
import plotly.express as px
#image reading
import tifffile as tf
from util import pil_imread
#import custom functions
from pre_processing import *

In [ ]:
def plot_2d_image(img_2d, zmax):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax,
        animation_frame=0
    )
    
    fig.show()

# Load image

In [ ]:
gen_path ="/groups/CaiLab/personal/Lex/raw/031322_11kgenes_experiment/notebook_pyfiles/dapi_aligned/fiducial_aligned"
tiff = tf.imread(gen_path + "/HybCycle_2/MMStack_Pos0.ome.tif")
#tiff = np.swapaxes(tiff,0,1)

# gen_path = "/groups/CaiLab/personal/Lex/raw/030922_11kgenes/notebook_pyfiles/dapi_aligned/final_background"
# tiff_bkgrd = tf.imread(gen_path + "/MMStack_Pos0.ome.tif")
# tiff_bkgrd = np.swapaxes(tiff_bkgrd,0,1)

In [ ]:
#check hyb image
plot_2d_image(tiff[0], zmax=20000)

# Background Correction

In [ ]:
# stack = raw image
# correction_algo = SigmaClipping_and_Gamma_C,Gaussian_and_Gamma_Correction, and LSR_Backgound_Correction
# stack_bkgrd = initial or final background image array
# z = number of z slices
# kern = kernel size for high pass gaussian
# size = image size
# gamma = gamma enhancment values
# sigma = sigma value for gaussian blurring
# match_hist = bool to match histograms of blurred image
# subtract = bool to subtract blurred image from raw (this works better if divide is set to false)
# divide = bool to divide blurred image from raw

corrected = background_correct_image(stack = tiff, correction_algo = Gaussian_and_Gamma_Correction, 
                                     stack_bkgrd=None, z=2, kern=5,
                                     size=2048, gamma=1.0, sigma=20,match_hist =False, 
                                     subtract=True, divide=False)

In [ ]:
plot_2d_image(corrected[0], zmax=10000)

In [ ]:
# corrected = background_correct_image(stack = tiff, correction_algo = Gaussian_and_Gamma_Correction,
#                                      stack_bkgrd=tiff_bkgrd, z=2, 
#                                      size=2048, gamma=1.4, sigma=40, match_hist =True, 
#                                      subtract=True, divide=False)

In [ ]:
#plot_2d_image(corrected[0], zmax=1000)

# RL Deconvolution

In [ ]:
# #good old Lucy
# #lb for leica boss and boc for box of chocolates
# rl_img_hpgb, psf = RL_deconvolution(corrected[:,:4,:,:], kern_rl=5, sigma=(1.8,1.6,1.5,1.3), 
#                    radius=(4,4,4,4), model="gaussian", microscope = "lb")

In [ ]:
# #deconvolved using a gaussian psf
# plot_2d_image(rl_img_hpgb[0], zmax=4000)

In [ ]:
# #good old Lucy
# #lb for leica boss and boc for box of chocolates
# rl_img_hpgb, psf = RL_deconvolution(corrected[:,:4,:,:], kern_rl=5, sigma=(1.8,1.6,1.5,1.3), 
#                    radius=(3,3,3,3), model="airy_disc", microscope = "lb")

In [ ]:
# #deconvolved using an airy disc psf
# plot_2d_image(rl_img_hpgb[0], zmax=1000)

In [ ]:
# #compare with non deconvolved
# plot_2d_image(corrected[0], zmax=800)

# Rolling Ball Subtraction

In [ ]:
#do you want to do rolling ball
rollingball = True
rb_radius = 5
corr_img = corrected 

#perform rolling ball
if rollingball == True:
    img_stack = []
    for z in range(corr_img.shape[0]):
        c_stack = []
        for c in range(corr_img.shape[1]):
            background = restoration.rolling_ball(corr_img[z][c], radius=rb_radius)
            rb_img = corr_img[z][c]-background
            rb_img[rb_img<0]=0
            c_stack.append(rb_img)
        img_stack.append(c_stack)
    corr_img = np.array(img_stack)

In [ ]:
corr_img.shape

In [ ]:
plot_2d_image(corr_img[0], zmax=1500)

# Low Pass Gaussian

In [ ]:
#lpgf
img_lpgb = low_pass_gaussian(corrected, kern = 3)

In [ ]:
plot_2d_image(img_lpgb[0], zmax=2000)

# Normalize Intensity

In [ ]:
#normalize int
scaled_img = scale_int(img_lpgb, p_min=80,p_max=99.99)

In [ ]:
plot_2d_image(scaled_img[0], zmax=0.5)

# Check sbatch

In [ ]:
#corrected
gen_path ="/groups/CaiLab/personal/Lex/raw/030922_11kgenes/notebook_pyfiles/dapi_aligned/pre_processed_images"
tiff = tf.imread(gen_path + "/HybCycle_0/MMStack_Pos0.ome.tif")

In [ ]:
tiff.shape

In [ ]:
plot_2d_image(tiff[0], zmax=1500)